In [8]:
import pandas as pd
import numpy as np
import torch
import pickle
import os
import json
import gc
from torch.distributions import Bernoulli
from torch.optim import LBFGS
from tqdm import tqdm
from scipy.stats import pearsonr
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager
import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tueplots import bundles
bundles.icml2024()

from torchmetrics import AUROC
auroc = AUROC(task="binary")

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(0)

device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

def visualize_response_matrix(results, value, filename):
    # Extract the groups labels in the order of the columns
    group_values = results.columns.get_level_values("scenario")

    # Identify the boundaries where the group changes
    boundaries = []
    for i in range(1, len(group_values)):
        if group_values[i] != group_values[i - 1]:
            boundaries.append(i - 0.5)  # using 0.5 to place the line between columns

    # Visualize the results with a matrix: red is 0, white is -1 and blue is 1
    cmap = mcolors.ListedColormap(["white", "red", "blue"])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    # Calculate midpoints for each group label
    groups_list = list(group_values)
    group_names = []
    group_midpoints = []
    current_group = groups_list[0]
    start_index = 0
    for i, grp in enumerate(groups_list):
        if grp != current_group:
            midpoint = (start_index + i - 1) / 2.0
            group_names.append(current_group)
            group_midpoints.append(midpoint)
            current_group = grp
            start_index = i
    # Add the last group
    midpoint = (start_index + len(groups_list) - 1) / 2.0
    group_names.append(current_group)
    group_midpoints.append(midpoint)

    # Define the minimum spacing between labels (e.g., 100 units)
    min_spacing = 100
    last_label_pos = -float("inf")
    # Plot the matrix
    with plt.rc_context(bundles.icml2024(usetex=True, family="serif")):
        fig, ax = plt.subplots(figsize=(20, 10))
        cax = ax.matshow(value, aspect="auto", cmap=cmap, norm=norm)

        # Add vertical lines at each boundary
        for b in boundaries:
            ax.axvline(x=b, color="black", linewidth=0.25, linestyle="--", alpha=0.5)
        
        # Add group labels above the matrix, only if they're spaced enough apart
        for name, pos in zip(group_names, group_midpoints):
            if pos - last_label_pos >= min_spacing:
                ax.text(pos, -5, name, ha='center', va='bottom', rotation=90, fontsize=3)
                last_label_pos = pos

        # Add model labels on the y-axis
        ax.set_yticks(range(len(results.index)))
        ax.set_yticklabels(results.index, fontsize=3)

        # Add a colorbar
        cbar = plt.colorbar(cax)
        cbar.set_ticks([-1, 0, 1])
        cbar.set_ticklabels(["-1", "0", "1"])
        plt.savefig(filename, dpi=600, bbox_inches="tight")
        plt.close()

def trainer(parameters, optim, closure, n_iter=100, verbose=True):
    pbar = tqdm(range(n_iter)) if verbose else range(n_iter)
    for iteration in pbar:
        if iteration > 0:
            previous_parameters = [p.clone() for p in parameters]
            previous_loss = loss.clone()
        
        loss = optim.step(closure)
        
        if iteration > 0:
            d_loss = (previous_loss - loss).item()
            d_parameters = sum(
                torch.norm(prev - curr, p=2).item()
                for prev, curr in zip(previous_parameters, parameters)
            )
            grad_norm = sum(torch.norm(p.grad, p=2).item() for p in parameters if p.grad is not None)
            if verbose:
                pbar.set_postfix({"grad_norm": grad_norm, "d_parameter": d_parameters, "d_loss": d_loss})
            
            if d_loss < 1e-5 and d_parameters < 1e-5 and grad_norm < 1e-5:
                break
    return parameters

def compute_auc(probs, data, train_idtor, test_idtor):
    train_probs = probs[train_idtor.bool()]
    test_probs = probs[test_idtor.bool()]
    train_labels = data[train_idtor.bool()]
    test_labels = data[test_idtor.bool()]
    train_auc = auroc(train_probs, train_labels)
    test_auc = auroc(test_probs, test_labels)
    print(f"train auc: {train_auc}")
    print(f"test auc: {test_auc}")
    
    return train_auc, test_auc

def compute_cttcorr(probs, data, train_idtor, test_idtor):
    train_probs  = probs.clone()
    test_probs   = probs.clone()
    train_labels = data.clone()
    test_labels  = data.clone()

    train_mask = ~train_idtor.bool()
    train_probs[train_mask]  = float('nan')
    train_labels[train_mask] = float('nan')

    test_mask = ~test_idtor.bool()
    test_probs[test_mask]   = float('nan')
    test_labels[test_mask]  = float('nan')
    
    train_prob_ctt = torch.nanmean(train_probs, dim=1).detach().cpu().numpy()
    train_label_ctt = torch.nanmean(train_labels, dim=1).detach().cpu().numpy()
    train_mask = ~np.isnan(train_prob_ctt) & ~np.isnan(train_label_ctt)
    train_cttcorr = pearsonr(train_prob_ctt[train_mask], train_label_ctt[train_mask]).statistic
    
    test_prob_ctt = torch.nanmean(test_probs, dim=1).detach().cpu().numpy()
    test_label_ctt = torch.nanmean(test_labels, dim=1).detach().cpu().numpy()
    test_mask = ~np.isnan(test_prob_ctt) & ~np.isnan(test_label_ctt)
    test_cttcorr = pearsonr(test_prob_ctt[test_mask], test_label_ctt[test_mask]).statistic
    
    print(f"train cttcorr: {train_cttcorr}")
    print(f"test cttcorr: {test_cttcorr}")

    return train_cttcorr, test_cttcorr

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.special import expit # Numerically stable sigmoid function
from scipy.stats import pearsonr

# --- PyTorch and Metrics Imports (from your reference code) ---
import torch
from torchmetrics import AUROC
auroc = AUROC(task="binary")
# --- End of Imports ---
# ===================================================================
# == Step 1: Load Data and Create Train/Test Split
# ===================================================================
print("Loading response matrix...")
resmat = pd.read_pickle("../data/resmat_2000.pkl")

Loading response matrix...


In [8]:
# Find the locations (row, col indices) of all non-missing values
non_nan_indices = np.argwhere(resmat.notna().values)

# Randomly shuffle these indices
np.random.seed(42)
np.random.shuffle(non_nan_indices)

# Decide on the split size
test_size = int(len(non_nan_indices) * 0.20)
test_indices = non_nan_indices[:test_size]
train_indices = non_nan_indices[test_size:]

# Create the training matrix by hiding the test data
train_resmat = resmat.copy()
test_rows, test_cols = train_resmat.values.shape[0], train_resmat.values.shape[1]
train_resmat.values[test_indices[:, 0], test_indices[:, 1]] = np.nan

print(f"Split data into {len(train_indices)} train samples and {len(test_indices)} test samples.")

# Impute the training data for SVD
imputed_train_resmat = train_resmat.fillna(0)

# ===================================================================
# == Step 2: Train SVD Model and Get Predictions
# ===================================================================
OPTIMAL_K = 8
print(f"\nTraining SVD model with k={OPTIMAL_K}...")

svd = TruncatedSVD(n_components=OPTIMAL_K, random_state=42)
svd.fit(imputed_train_resmat)

# Reconstruct the full matrix and convert to probabilities
reconstructed_matrix = svd.inverse_transform(svd.transform(imputed_train_resmat))
probs_matrix_np = expit(reconstructed_matrix)

# ===================================================================
# == Step 3: Prepare Data for Evaluation (NumPy and PyTorch)
# ===================================================================
# Ground truth data as a numpy array
data_np = resmat.fillna(0).values

# Create boolean masks (numpy)
train_idtor_np = np.zeros_like(data_np, dtype=bool)
test_idtor_np = np.zeros_like(data_np, dtype=bool)
train_idtor_np[train_indices[:, 0], train_indices[:, 1]] = True
test_idtor_np[test_indices[:, 0], test_indices[:, 1]] = True

# --- NEW: Convert NumPy arrays to PyTorch tensors for compatibility ---
print("\nConverting results to PyTorch Tensors for evaluation...")
device = "cuda:0" if torch.cuda.is_available() else "cpu"

probs_tensor = torch.tensor(probs_matrix_np, dtype=torch.float32, device=device)
data_tensor = torch.tensor(data_np, dtype=torch.float32, device=device)
train_idtor_tensor = torch.tensor(train_idtor_np, dtype=torch.int, device=device)
test_idtor_tensor = torch.tensor(test_idtor_np, dtype=torch.int, device=device)
# --- End of Conversion ---

print("\n--- Running Final Evaluations ---")
compute_auc(probs_tensor, data_tensor, train_idtor_tensor, test_idtor_tensor)
compute_cttcorr(probs_tensor, data_tensor, train_idtor_tensor, test_idtor_tensor)

Split data into 910536 train samples and 227634 test samples.

Training SVD model with k=8...

Converting results to PyTorch Tensors for evaluation...

--- Running Final Evaluations ---
train auc: 0.9263309240341187
test auc: 0.8913356065750122
train cttcorr: 0.9637015461921692
test cttcorr: 0.9530106782913208


(np.float32(0.96370155), np.float32(0.9530107))

In [4]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import TruncatedSVD
from scipy.special import expit
from scipy.stats import pearsonr
from torchmetrics import AUROC
# ===================================================================
# == Step 1: Create a Single, Unified Train/Test Split
# ===================================================================
print("--- Step 1: Creating a single cell-level train/test split ---")
resmat = pd.read_pickle("../data/resmat_2000.pkl")
device = "cuda" if torch.cuda.is_available() else "cpu"

data_withnan = torch.tensor(resmat.values, dtype=torch.float32)
data_idtor = torch.isfinite(data_withnan) # Mask of all known values

valid_condition = False
trial = 0
while not valid_condition:
    train_idtor = torch.bernoulli(data_idtor.float() * 0.8).int()
    test_idtor = data_idtor.int() - train_idtor
    # Ensure no row or column in the training set is entirely empty
    valid_condition = (train_idtor.sum(dim=1) > 0).all() and (train_idtor.sum(dim=0) > 0).all()
    print(f"  Trial {trial}: Valid split condition met: {valid_condition}")
    trial += 1


--- Step 1: Creating a single cell-level train/test split ---
  Trial 0: Valid split condition met: True


In [5]:
# ===================================================================
# == Step 2: Train SVD *ONLY* on the Training Set
# ===================================================================
print("\n--- Step 2: Training SVD on the training set ONLY ---")
# Create a training matrix by hiding the test values
train_resmat_df = resmat.copy()
train_resmat_df[test_idtor.bool().numpy()] = np.nan
imputed_train_resmat = train_resmat_df.fillna(0)

OPTIMAL_K = 8
svd_model = TruncatedSVD(n_components=OPTIMAL_K, random_state=42)
# Fit the SVD model ONLY on the training data
svd_model.fit(imputed_train_resmat)

# Get the item parameters (y target for NN) from this training-only model
svd_item_params = svd_model.components_
# Get the model abilities by transforming the FULL imputed data
svd_model_abilities = svd_model.transform(resmat.fillna(0).values)



--- Step 2: Training SVD on the training set ONLY ---


In [9]:
# ===================================================================
# == Step 3: Train NN to Predict Item Parameters
# ===================================================================
print("\n--- Step 3: Training Neural Network to predict item parameters ---")

with open("../data/embed_meta-llama_Llama-3.1-8B-Instruct.pkl", "rb") as f:
    df_embed = pickle.load(f)

question_to_emb = dict(zip(df_embed["question"], df_embed["embedding"])); questions = resmat.columns.get_level_values("input.text").tolist()
embeds = [question_to_emb.get(q, None) for q in questions]
# The target abilities are from the SVD trained ONLY on the training set
y_abilities_all = svd_item_params.T

combined_data = pd.DataFrame({'embedding': embeds, 'ability_vector': list(y_abilities_all)})
cleaned_data = combined_data.dropna(subset=['embedding'])
X_embeddings = np.vstack(cleaned_data['embedding'].values)
y_abilities = np.vstack(cleaned_data['ability_vector'].values)


--- Step 3: Training Neural Network to predict item parameters ---


In [16]:
class AbilityPredictor(nn.Module):
    def __init__(self, e, n): 
        super(AbilityPredictor, self).__init__();
        self.network = nn.Sequential(nn.Linear(e, 1024),
                                     nn.ReLU(), nn.Dropout(0.5),
                                     nn.Linear(1024, 512),
                                     nn.ReLU(), nn.Dropout(0.5),
                                     nn.Linear(512, 128),
                                     nn.ReLU(),
                                     nn.Dropout(0.5),
                                     nn.Linear(128, n))
    def forward(self, x): return self.network(x)

# Note: We train the NN on ALL available items, because the target (y_abilities)
# was already derived purely from the training set, so there is no data leakage.
X_train_t = torch.tensor(X_embeddings, dtype=torch.float32)
y_train_t = torch.tensor(y_abilities, dtype=torch.float32)
train_dataset = TensorDataset(X_train_t, y_train_t); train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
nn_model = AbilityPredictor(X_embeddings.shape[1], y_abilities.shape[1]); nn_model.to(device)
loss_fn = nn.MSELoss(); optimizer = torch.optim.Adam(nn_model.parameters(), lr=0.0001)

n_epochs = 200
for epoch in range(n_epochs):
    nn_model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        p = nn_model(X_batch); loss = loss_fn(p, y_batch)
        optimizer.zero_grad(); loss.backward(); optimizer.step(); total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{n_epochs} | Average Training Loss: {avg_loss:.6f}")
print("--- NN Training Complete ---")

# ===================================================================
# == Step 4: Reconstruct and Evaluate with the Unified Split
# ===================================================================
print("\n--- Step 4: Reconstructing matrix and running final evaluation ---")

embed_dim = X_embeddings.shape[1]
full_embeds_np = np.zeros((len(embeds), embed_dim))
for i, emb in enumerate(embeds):
    if emb is not None: full_embeds_np[i] = emb

nn_model.eval()
with torch.no_grad():
    input_tensor = torch.tensor(full_embeds_np, dtype=torch.float32).to(device)
    nn_predicted_item_params = nn_model(input_tensor).cpu().numpy()

# Reconstruct the logit matrix
reconstructed_matrix_from_nn = svd_model_abilities @ nn_predicted_item_params.T
# Convert logits to probabilities
probs_matrix_from_nn = expit(reconstructed_matrix_from_nn)

# Prepare all final tensors for evaluation
probs_tensor = torch.tensor(probs_matrix_from_nn, dtype=torch.float32, device=device)
data_tensor = torch.tensor(resmat.fillna(0).values, dtype=torch.float32, device=device)
train_idtor_tensor = train_idtor.to(device)
test_idtor_tensor = test_idtor.to(device)

# Run the final evaluations using the unified train/test split
print("\n--- Final Evaluation of NN-based Reconstruction ---")
compute_auc(probs_tensor, data_tensor, train_idtor_tensor, test_idtor_tensor)
compute_cttcorr(probs_tensor, data_tensor, train_idtor_tensor, test_idtor_tensor)

Epoch 1/200 | Average Training Loss: 0.000435
Epoch 2/200 | Average Training Loss: 0.000160
Epoch 3/200 | Average Training Loss: 0.000110
Epoch 4/200 | Average Training Loss: 0.000083
Epoch 5/200 | Average Training Loss: 0.000067
Epoch 6/200 | Average Training Loss: 0.000056
Epoch 7/200 | Average Training Loss: 0.000049
Epoch 8/200 | Average Training Loss: 0.000044
Epoch 9/200 | Average Training Loss: 0.000039
Epoch 10/200 | Average Training Loss: 0.000037
Epoch 11/200 | Average Training Loss: 0.000034
Epoch 12/200 | Average Training Loss: 0.000033
Epoch 13/200 | Average Training Loss: 0.000032
Epoch 14/200 | Average Training Loss: 0.000031
Epoch 15/200 | Average Training Loss: 0.000030
Epoch 16/200 | Average Training Loss: 0.000030
Epoch 17/200 | Average Training Loss: 0.000029
Epoch 18/200 | Average Training Loss: 0.000029
Epoch 19/200 | Average Training Loss: 0.000028
Epoch 20/200 | Average Training Loss: 0.000028
Epoch 21/200 | Average Training Loss: 0.000027
Epoch 22/200 | Average

(np.float32(0.94698703), np.float32(0.9429172))

In [17]:
# After the training loop is finished
torch.save(nn_model.state_dict(), '../result/final_ability_predictor_model.pth')
print("\n--- Final model saved to final_ability_predictor_model.pth ---")


--- Final model saved to final_ability_predictor_model.pth ---
